### 线性规划与Word Mover's Distance

WMD在文本分析领域算作是一个比较经典的算法，它可以用来计算两个文本之间的相似度。 比如问答系统中，可以判断一个用户的query跟哪一个知识库里的问题最相近。而且，计算两个文本之间的相似度这个问题是NLP的核心，这也是为什么文本相似度计算这么重要的原因。 

背景： 在文本相似度匹配问题上如果使用tf-idf等模型，那这时候假如两个文本中没有出现共同的单词，则计算出来的相似度为0，但我们知道实际上很多时候单词可能不一样，但表示的内容确是类似的。 比如 ”People like this car“, "Those guys enjoy driving that", 虽然没有任何一样的单词，意思确是类似的。 这是WMD算法提出来的初衷。

WMD作为文本相似度计算的一种方法，最早由Matt J. Kusner, Yu Sun, Nicholas I. Kolkin, Kilian Q. Weinberger等人提出。但实际上它的想法极其简单，可以认为是Transportation Problem用在了词向量上， 其核心是线性规划。 对于Transportation问题在课上已经讲过，仍不清楚的朋友可以回顾一下课程的内容。 

#### 1. WMD算法的实现
具体算法的实现是基于线性规划问题，细节请参考[WMD的论文](http://proceedings.mlr.press/v37/kusnerb15.pdf)或[博客](https://adline125.github.io/2020/06/04/WMD-From-Word-Embeddings-To-Document-Distances/#more)。 核心思想是把第一个句子转换成第二个句子过程中需要花费的最小cost。 

<img src="picture1.png" alt="drawing" width="600"/>


线性规划问题即可以写成如下形式：

$$
\begin{split}
\min _ { T \geq 0 } &\sum _ { i , j = 1 } ^ { n } T _ { i j } c ( i , j ) \\
subject\ to: &\sum _ { j = 1 } ^ { n } T _ { i j } = d _ { i } \quad \forall i \in \{ 1 , \ldots , n \} \qquad (1)\\
&\sum _ { i = 1 } ^ { n } T _ { i j } = d _ { j } ^ { \prime } \quad \forall j \in \{ 1 , \ldots , n \} \qquad (2)
\end{split} \\
$$



这里的参数是$T_{ij}$， 需要通过LP Solver去解决。$c(i,j)$指的是两个单词之间的距离, $c_{i,j}=||x_i-x_j||_2$。 参考： $||x||_2=\sqrt{x_1^2+...+x_d^2}$

为了实现WMD算法，首先需要词向量。 可以直接使用已经训练好的词向量。 
请下载训练好的Glove向量：https://nlp.stanford.edu/projects/glove/， 本例下载其中的 glove.6B.zip， 并使用d=100维的向量。 

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.metrics import euclidean_distances
from cvxopt import matrix, solvers

# 读取Glove文件。 注意： 不要试图修改文件以及路径
glovefile = open("glove.6B.100d.txt","r",encoding="utf-8")
glove_model = {}
lines = glovefile.readlines()
for line in lines:
    glove_model[line[0: line.index(' ')]] = line[line.index(' ') + 1:].split(' ')

# TODO: 编写WMD函数来计算两个句子之间的相似度
def sent_vectorize(sent1, sent2):
    vectorizer = CountVectorizer(stop_words='english')
    vect = vectorizer.fit_transform([sent1, sent2])
    norm_vect = normalize(vect.toarray(), axis=1, norm='l1')
    vocab = np.array(vectorizer.get_feature_names())
    return vocab, norm_vect


def WMD(sent1, sent2):
    """
    参数sent1是第一个句子， 参数sent2是第二个句子

    在实现WMD算法的时候，需要用到LP Solver用来解决Transportation proboem. 请使用http://cvxopt.org/examples/tutorial/lp.html
    也可以参考blog： https://scaron.info/blog/linear-programming-in-python-with-cvxopt.html

    需要做的事情为：

    1. 把两个句子表示成论文中的nBow
    2. 获取每个单词的词向量。这需要读取文件之后构建embedding matrix.
    3. 构建lp问题，并用solver解决
    """
    # 获取词典，以及句子的归一化词集模型向量
    vocab, norm_vect = sent_vectorize(sent1, sent2)
    # 分别去掉句子中的停用词
    sent1_words = vocab[np.nonzero(norm_vect[0])]
    sent2_words = vocab[np.nonzero(norm_vect[1])]
    # 获取句子中每个单词的词向量
    sent1_word2vec = []
    sent2_word2vec = []
    for word in sent1_words:
        sent1_word2vec.append(glove_model[word])
    for word in sent2_words:
        sent2_word2vec.append(glove_model[word])

    # 计算sent1中的各个词向量与sent2中的各个词向量俩俩之间的欧式距离
    C_tmp = euclidean_distances(np.array(sent1_word2vec), np.array(sent2_word2vec))
    C = C_tmp.flatten().astype(np.double)  # 相当于运输单价

    n = len(C)  # n个变量
    # 根据归一化的词集模型向量得到对应的约束条件常数项的向量,分别对应供给量和需求量
    sent1_vec = norm_vect[0][norm_vect[0] != 0]
    sent2_vec = norm_vect[1][norm_vect[1] != 0]

    b_2 = np.zeros(n)  #变量均大或等于0的约束条件
    b_1 = np.concatenate((sent1_vec, sent2_vec))  # 相当于lp中的b
    b = []
    for pos, neg in zip(b_1, -b_1):  # 把等式约束条件改写成两个不等式约束条件，即同时满足大于等于和小于等于d, 相当于等于d
        b.append(pos)
        b.append(neg)
    b = np.concatenate((np.array(b), b_2))  # 供给量和需求量约束条件以及变量均大或等于0的约束条件

    # 生成约束条件的系数矩阵A
    m1 = len(sent1_vec)
    m2 = len(sent2_vec)
    m = m1 + m2  # 2m个约束条件，因为这里的等式约束拆分成了两个不等式约束
    A = np.zeros((2 * (m1 + m2), m1 * m2), dtype=np.double)  # 行为两个m1对应两个di不等式两个m2对应两个dj不等式；列为各不等式对应的Tij
    for i in range(m1):  # 对于等号右边为di的等式转变成的两个不等式
        for j in range(m2):
            A[2 * i][j + i * m2] = 1  # 偶数行是小于等于di的不等式
            A[2 * i + 1][j + i * m2] = -1  # 奇数行是大于等于di的不等式

    for i in range(m2):
        for j in range(m1):  # 对于等号右边为dj的等式转变成的两个不等式
            # A[m1+i][i+j*m2]=1
            A[2 * m1 + 2 * i][i + j * m2] = 1  # 偶数行是小于等于dj的不等式
            A[2 * m1 + 2 * i + 1][i + j * m2] = -1  # 奇数行是大于等于dj的不等式

    G_1 = np.diag([-1] * n)  # #变量均大或等于0的约束条件
    G = np.concatenate((A, G_1))

    # 转换成cvxopt的矩阵形式
    c = matrix(C)
    G = matrix(G)
    b = matrix(b)

    # lp 问题求解
    solvers.options['show_progress'] = False
    sol = solvers.lp(c, G, b, solver='glpk')
    wmd_dist = sol['primal objective']  # 最优目标值

    return wmd_dist
    

In [5]:
print(WMD("Obama speaks to the media in Illinois","The President greets the press in Chicago"))
print(WMD("The band gave a concert in Japan","The President greets the press in Chicago"))
print(WMD("Obama speaks in Illinois","The President greets the press in Chicago"))
print (WMD("people like this car", "those guys enjoy driving that"))
print (WMD("people like this car", "those guys enjoy drinking that"))

5.004112166198258
6.517790677948749
5.607367653449974
4.746836132454052
5.907876495463484
